<a href="https://colab.research.google.com/github/VasaviLagishetty/COVID19/blob/master/COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import requests
from datetime import datetime

In [2]:
raw= requests.get("https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/1/query?where=1%3D1&outFields=*&outSR=4326&f=json")
raw_json = raw.json()
df = pd.DataFrame(raw_json["features"])
df.head()

,attributes,geometry
0,"{'OBJECTID': 1, 'Province_State': 'Abruzzo', '...","{'x': 13.398438230000068, 'y': 42.35122196000003}"
1,"{'OBJECTID': 2, 'Province_State': 'Acre', 'Cou...","{'x': -70.81199999999995, 'y': -9.023799999999..."
2,"{'OBJECTID': 3, 'Province_State': 'Adygea Repu...","{'x': 40.152042100000074, 'y': 44.69390060000006}"
3,"{'OBJECTID': 4, 'Province_State': 'Aguascalien...","{'x': -102.29159999999996, 'y': 21.88530000000..."
4,"{'OBJECTID': 5, 'Province_State': 'Aichi', 'Co...","{'x': 137.21162100000004, 'y': 35.035551000000..."


In [3]:
df["attributes"][0]

{'Active': 106,
 'Admin2': None,
 'Combined_Key': 'Abruzzo, Italy',
 'Confirmed': 3377,
 'Country_Region': 'Italy',
 'Deaths': 472,
 'FIPS': None,
 'Last_Update': 1596206138000,
 'Lat': 42.35122196,
 'Long_': 13.39843823,
 'OBJECTID': 1,
 'Province_State': 'Abruzzo',
 'Recovered': 2799}

In [4]:
df['geometry'][0]

{'x': 13.398438230000068, 'y': 42.35122196000003}

In [5]:
data_list = df["attributes"].tolist()
data = pd.DataFrame(data_list)
data = data.set_index("OBJECTID")
data = data.reset_index(drop = True)
data

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths,Active,Admin2,FIPS,Combined_Key
0,Abruzzo,Italy,1.596206e+12,42.351222,13.398438,3377,2799,472,106,None,None,"Abruzzo, Italy"
1,Acre,Brazil,1.596206e+12,-9.023800,-70.812000,19573,13724,521,5328,None,None,"Acre, Brazil"
2,Adygea Republic,Russia,1.596206e+12,44.693901,40.152042,2747,2112,20,615,None,None,"Adygea Republic, Russia"
3,Aguascalientes,Mexico,1.596206e+12,21.885300,-102.291600,3983,3239,253,491,None,None,"Aguascalientes, Mexico"
4,Aichi,Japan,1.596206e+12,35.035551,137.211621,1799,551,35,1213,None,None,"Aichi, Japan"
...,...,...,...,...,...,...,...,...,...,...,...,...
740,Ohio,US,1.596206e+12,40.388783,-82.764915,89626,0,3442,86184,None,39,"Ohio, US"
741,Alabama,US,1.596206e+12,32.318200,-86.902300,85762,0,1565,84197,None,01,"Alabama, US"
742,North Carolina,US,1.596206e+12,35.630066,-79.806419,120532,0,1922,118610,None,37,"North Carolina, US"
743,District of Columbia,US,1.596206e+12,38.897438,-77.026817,12057,0,584,11473,None,11,"District of Columbia, US"


In [6]:
bool_series = pd.isnull(data['Last_Update']) 
data[bool_series]

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths,Active,Admin2,FIPS,Combined_Key
699,American Samoa,US,NaN,-14.271,-170.132,0,0,0,0,None,None,"American Samoa, US"
710,United States Virgin Islands,US,NaN,18.350,-64.933,0,0,0,0,None,None,"United States Virgin Islands, US"


In [7]:
data = data[data['Last_Update'].notna()]
data["Province_State"].fillna(value="", inplace=True)

data

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths,Active,Admin2,FIPS,Combined_Key
0,Abruzzo,Italy,1.596206e+12,42.351222,13.398438,3377,2799,472,106,None,None,"Abruzzo, Italy"
1,Acre,Brazil,1.596206e+12,-9.023800,-70.812000,19573,13724,521,5328,None,None,"Acre, Brazil"
2,Adygea Republic,Russia,1.596206e+12,44.693901,40.152042,2747,2112,20,615,None,None,"Adygea Republic, Russia"
3,Aguascalientes,Mexico,1.596206e+12,21.885300,-102.291600,3983,3239,253,491,None,None,"Aguascalientes, Mexico"
4,Aichi,Japan,1.596206e+12,35.035551,137.211621,1799,551,35,1213,None,None,"Aichi, Japan"
...,...,...,...,...,...,...,...,...,...,...,...,...
740,Ohio,US,1.596206e+12,40.388783,-82.764915,89626,0,3442,86184,None,39,"Ohio, US"
741,Alabama,US,1.596206e+12,32.318200,-86.902300,85762,0,1565,84197,None,01,"Alabama, US"
742,North Carolina,US,1.596206e+12,35.630066,-79.806419,120532,0,1922,118610,None,37,"North Carolina, US"
743,District of Columbia,US,1.596206e+12,38.897438,-77.026817,12057,0,584,11473,None,11,"District of Columbia, US"


In [8]:
def convertTime(t):
  t = int(t)
  return datetime.fromtimestamp(t)

data['Last_Update'] = data['Last_Update']/1000
data['Last_Update'] = data['Last_Update'].apply(convertTime)
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths,Active,Admin2,FIPS,Combined_Key
0,Abruzzo,Italy,2020-07-31 14:35:38,42.351222,13.398438,3377,2799,472,106,None,None,"Abruzzo, Italy"
1,Acre,Brazil,2020-07-31 14:35:38,-9.023800,-70.812000,19573,13724,521,5328,None,None,"Acre, Brazil"
2,Adygea Republic,Russia,2020-07-31 14:35:38,44.693901,40.152042,2747,2112,20,615,None,None,"Adygea Republic, Russia"
3,Aguascalientes,Mexico,2020-07-31 14:35:38,21.885300,-102.291600,3983,3239,253,491,None,None,"Aguascalientes, Mexico"
4,Aichi,Japan,2020-07-31 14:35:38,35.035551,137.211621,1799,551,35,1213,None,None,"Aichi, Japan"
...,...,...,...,...,...,...,...,...,...,...,...,...
740,Ohio,US,2020-07-31 14:35:38,40.388783,-82.764915,89626,0,3442,86184,None,39,"Ohio, US"
741,Alabama,US,2020-07-31 14:35:38,32.318200,-86.902300,85762,0,1565,84197,None,01,"Alabama, US"
742,North Carolina,US,2020-07-31 14:35:38,35.630066,-79.806419,120532,0,1922,118610,None,37,"North Carolina, US"
743,District of Columbia,US,2020-07-31 14:35:38,38.897438,-77.026817,12057,0,584,11473,None,11,"District of Columbia, US"


In [9]:
df_total = data.groupby('Country_Region',as_index=False).agg({'Confirmed' : 'sum','Recovered' : 'sum','Deaths' : 'sum'}) 

In [10]:
df_total

,Country_Region,Confirmed,Recovered,Deaths
0,Afghanistan,36675,25509,1272
1,Albania,5197,2883,154
2,Algeria,29831,20082,1200
3,Andorra,925,807,52
4,Angola,1109,395,51
...,...,...,...,...
183,West Bank and Gaza,11837,5077,82
184,Western Sahara,10,8,1
185,Yemen,1726,856,487
186,Zambia,5963,3803,151


In [11]:
total_confirmed = data['Confirmed'].sum()
total_recovered = data['Recovered'].sum()
total_deaths = data['Deaths'].sum()
print("Total confirmed cases till",data['Last_Update'][0],"are",total_confirmed)
print("Total recovered cases till",data['Last_Update'][0],"are",total_recovered)
print("Total death cases till",data['Last_Update'][0],"are",total_deaths)

Total confirmed cases till 2020-07-31 14:35:38 are 17334539
Total recovered cases till 2020-07-31 14:35:38 are 10179718
Total death cases till 2020-07-31 14:35:38 are 674038


In [12]:
df_top10 = df_total.nlargest(10, "Confirmed")
top10_countries_1 = df_top10["Country_Region"].tolist()
top10_confirmed = df_top10["Confirmed"].tolist()

df_top10 = df_total.nlargest(10, "Recovered")
top10_countries_2 = df_top10["Country_Region"].tolist()
top10_recovered = df_top10["Recovered"].tolist()

df_top10 = df_total.nlargest(10, "Deaths")
top10_countries_3 = df_top10["Country_Region"].tolist()
top10_deaths = df_top10["Deaths"].tolist()

# Subplots

In [13]:
fig = make_subplots(
    rows = 4, cols = 6,
    specs=[
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"} ],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,              {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
          ]
)

In [14]:
message = data["Country_Region"] + " " + data["Province_State"] + "<br>"
message += "Confirmed: " + data["Confirmed"].astype(str) + "<br>"
message += "Deaths: " + data["Deaths"].astype(str) + "<br>"
message += "Recovered: " + data["Recovered"].astype(str) + "<br>"
message += "Last updated: " + data["Last_Update"].astype(str)
data.loc[:,'text'] = message
data

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths,Active,Admin2,FIPS,Combined_Key,text
0,Abruzzo,Italy,2020-07-31 14:35:38,42.351222,13.398438,3377,2799,472,106,None,None,"Abruzzo, Italy",Italy Abruzzo<br>Confirmed: 3377<br>Deaths: 47...
1,Acre,Brazil,2020-07-31 14:35:38,-9.023800,-70.812000,19573,13724,521,5328,None,None,"Acre, Brazil",Brazil Acre<br>Confirmed: 19573<br>Deaths: 521...
2,Adygea Republic,Russia,2020-07-31 14:35:38,44.693901,40.152042,2747,2112,20,615,None,None,"Adygea Republic, Russia",Russia Adygea Republic<br>Confirmed: 2747<br>D...
3,Aguascalientes,Mexico,2020-07-31 14:35:38,21.885300,-102.291600,3983,3239,253,491,None,None,"Aguascalientes, Mexico",Mexico Aguascalientes<br>Confirmed: 3983<br>De...
4,Aichi,Japan,2020-07-31 14:35:38,35.035551,137.211621,1799,551,35,1213,None,None,"Aichi, Japan",Japan Aichi<br>Confirmed: 1799<br>Deaths: 35<b...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,Ohio,US,2020-07-31 14:35:38,40.388783,-82.764915,89626,0,3442,86184,None,39,"Ohio, US",US Ohio<br>Confirmed: 89626<br>Deaths: 3442<br...
741,Alabama,US,2020-07-31 14:35:38,32.318200,-86.902300,85762,0,1565,84197,None,01,"Alabama, US",US Alabama<br>Confirmed: 85762<br>Deaths: 1565...
742,North Carolina,US,2020-07-31 14:35:38,35.630066,-79.806419,120532,0,1922,118610,None,37,"North Carolina, US",US North Carolina<br>Confirmed: 120532<br>Deat...
743,District of Columbia,US,2020-07-31 14:35:38,38.897438,-77.026817,12057,0,584,11473,None,11,"District of Columbia, US",US District of Columbia<br>Confirmed: 12057<br...


In [15]:
fig.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        lon = data["Long_"],
        lat = data["Lat"],
        hovertext = data["text"],
        showlegend=False,
        marker = dict(
            size = 5,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = True,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(50, 50, 50)'
            ),
            cmin = 0,
            color = data['Confirmed'],
            cmax = data['Confirmed'].max(),
            colorbar_title="Confirmed Cases<br>Latest Update",  
            colorbar_x = -0.05
        )

    ),    
    row=1, col=1
)

fig.update_geos(
    projection_type="natural earth",
    landcolor="grey",
    oceancolor="LightBlue",
    showocean=True,
    lakecolor="LightBlue"
)



In [16]:
# Confirmed Cases
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_confirmed,
        title="Confirmed Cases",
    ),
    row=1, col=4
)

# Recovered Cases
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_recovered,
        title="Recovered Cases",
    ),
    row=1, col=5
)

# Death Cases
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_deaths,
        title="Deaths Cases",
    ),
    row=1, col=6
)

In [17]:
fig.add_trace(
    go.Bar(
        x=top10_countries_1,
        y=top10_confirmed, 
        name= "Confirmed Cases",
        marker=dict(color="Yellow"), 
        showlegend=True,
    ),
    row=2, col=4
)

fig.add_trace(
    go.Bar(
        x=top10_countries_2,
        y=top10_recovered, 
        name= "Recovered Cases",
        marker=dict(color="Green"), 
        showlegend=True),
    row=3, col=4
)

fig.add_trace(
    go.Bar(
        x=top10_countries_3,
        y=top10_deaths, 
        name= "Deaths Cases",
        marker=dict(color="crimson"), 
        showlegend=True),
    row=4, col=4
)

In [18]:
fig.update_layout(
    template="plotly_dark",
    title = "Global COVID-19 Cases (Last Updated: " + str(data["Last_Update"][0]) + ")",
    showlegend=True,
    legend_orientation="h",
    legend=dict(x=0.65, y = -0.1),
    font=dict(family='Candara', size=12, color='#FFFFFF')
)

In [19]:
# Saving to a HTML file
fig.write_html('COVID19.html', auto_open=True)

[For updation of data from kaggle](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset/data?select=covid_19_data.csv)

In [20]:
url = "https://raw.githubusercontent.com/VasaviLagishetty/COVID19/master/covid_19_data.csv"
dataset = pd.read_csv(url)

In [21]:
dataset

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
39342,39343,06/10/2020,Zacatecas,Mexico,2020-06-11 03:33:41,423.0,53.0,266.0
39343,39344,06/10/2020,Zakarpattia Oblast,Ukraine,2020-06-11 03:33:41,1448.0,33.0,628.0
39344,39345,06/10/2020,Zaporizhia Oblast,Ukraine,2020-06-11 03:33:41,502.0,15.0,296.0
39345,39346,06/10/2020,Zhejiang,Mainland China,2020-06-11 03:33:41,1268.0,1.0,1267.0


In [22]:
dataset["Country/Region"].nunique()

223

In [23]:
df = dataset.groupby(dataset["Country/Region"]).sum()
df = df.drop(['SNo'], axis = 1)
df = df.sort_values(by='Confirmed', ascending=False)
df.reset_index(drop=False, inplace=True)
df = df.head(10)
df 

,Country/Region,Confirmed,Deaths,Recovered
0,US,84900037.0,4765164.0,15449638.0
1,Spain,15609455.0,1708931.0,8110890.0
2,Brazil,15540416.0,926913.0,6553166.0
3,Italy,15399969.0,2072220.0,6760057.0
4,Russia,13936698.0,148489.0,4523988.0
5,UK,13363760.0,1860786.0,62165.0
6,France,11852385.0,1644423.0,3574872.0
7,Germany,11810824.0,448692.0,8608350.0
8,Mainland China,10047149.0,453792.0,7906544.0
9,Turkey,8366809.0,221513.0,4884878.0


## Top 10 Countries with Confirmed Cases

In [24]:
import plotly.graph_objects as go

cases = go.Figure(go.Bar(
            x=df['Confirmed'],
            y=df['Country/Region'],
            orientation='h'))

cases.show()

In [25]:
India_covid_data = dataset[dataset['Country/Region'] == "India"]
India_covid_data

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
430,431,01/30/2020,NaN,India,1/30/20 16:00,1.0,0.0,0.0
491,492,01/31/2020,NaN,India,1/31/2020 23:59,1.0,0.0,0.0
547,548,02/01/2020,NaN,India,1/31/2020 8:15,1.0,0.0,0.0
607,608,02/02/2020,NaN,India,2020-02-02T06:03:08,2.0,0.0,0.0
672,673,02/03/2020,NaN,India,2020-02-03T21:43:02,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...
39281,39282,06/10/2020,Tripura,India,2020-06-11 03:33:41,864.0,1.0,192.0
39295,39296,06/10/2020,Unknown,India,2020-06-11 03:33:41,9227.0,0.0,0.0
39301,39302,06/10/2020,Uttar Pradesh,India,2020-06-11 03:33:41,11335.0,301.0,6669.0
39302,39303,06/10/2020,Uttarakhand,India,2020-06-11 03:33:41,1537.0,13.0,755.0


In [26]:
India_covid_data = India_covid_data.drop(['SNo','Province/State',],axis=1)
India_covid_data

,ObservationDate,Country/Region,Last Update,Confirmed,Deaths,Recovered
430,01/30/2020,India,1/30/20 16:00,1.0,0.0,0.0
491,01/31/2020,India,1/31/2020 23:59,1.0,0.0,0.0
547,02/01/2020,India,1/31/2020 8:15,1.0,0.0,0.0
607,02/02/2020,India,2020-02-02T06:03:08,2.0,0.0,0.0
672,02/03/2020,India,2020-02-03T21:43:02,3.0,0.0,0.0
...,...,...,...,...,...,...
39281,06/10/2020,India,2020-06-11 03:33:41,864.0,1.0,192.0
39295,06/10/2020,India,2020-06-11 03:33:41,9227.0,0.0,0.0
39301,06/10/2020,India,2020-06-11 03:33:41,11335.0,301.0,6669.0
39302,06/10/2020,India,2020-06-11 03:33:41,1537.0,13.0,755.0


In [27]:
India_covid_data.dtypes

ObservationDate     object
Country/Region      object
Last Update         object
Confirmed          float64
Deaths             float64
Recovered          float64
dtype: object

In [28]:
cols = India_covid_data.columns[India_covid_data.dtypes.eq('float64')]
India_covid_data[cols] = India_covid_data[cols].astype(int)
India_covid_data.rename(columns={'Country/Region' : 'Country','ObservationDate':'Date'},inplace=True)
India_covid_data

,Date,Country,Last Update,Confirmed,Deaths,Recovered
430,01/30/2020,India,1/30/20 16:00,1,0,0
491,01/31/2020,India,1/31/2020 23:59,1,0,0
547,02/01/2020,India,1/31/2020 8:15,1,0,0
607,02/02/2020,India,2020-02-02T06:03:08,2,0,0
672,02/03/2020,India,2020-02-03T21:43:02,3,0,0
...,...,...,...,...,...,...
39281,06/10/2020,India,2020-06-11 03:33:41,864,1,192
39295,06/10/2020,India,2020-06-11 03:33:41,9227,0,0
39301,06/10/2020,India,2020-06-11 03:33:41,11335,301,6669
39302,06/10/2020,India,2020-06-11 03:33:41,1537,13,755


In [29]:
India_covid_data['Date'] = pd.to_datetime(India_covid_data['Date'])
India_covid_data['Date'] = India_covid_data['Date'].dt.strftime('%Y-%m-%d')
India_covid_data

,Date,Country,Last Update,Confirmed,Deaths,Recovered
430,2020-01-30,India,1/30/20 16:00,1,0,0
491,2020-01-31,India,1/31/2020 23:59,1,0,0
547,2020-02-01,India,1/31/2020 8:15,1,0,0
607,2020-02-02,India,2020-02-02T06:03:08,2,0,0
672,2020-02-03,India,2020-02-03T21:43:02,3,0,0
...,...,...,...,...,...,...
39281,2020-06-10,India,2020-06-11 03:33:41,864,1,192
39295,2020-06-10,India,2020-06-11 03:33:41,9227,0,0
39301,2020-06-10,India,2020-06-11 03:33:41,11335,301,6669
39302,2020-06-10,India,2020-06-11 03:33:41,1537,13,755


In [30]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=India_covid_data['Date'], y=India_covid_data['Confirmed'],
                    mode='lines',
                    name='confirmed'))

fig.add_trace(go.Scatter(x=India_covid_data['Date'], y=India_covid_data['Deaths'],
                    mode='lines',
                    name='deaths'))

fig.add_trace(go.Scatter(x=India_covid_data['Date'], y=India_covid_data['Recovered'],
                    mode='lines',
                    name='recovered'))

fig.update_layout(
    template="plotly_dark",
    title="INDIA COVID19 CASES",
    xaxis_title="Date",
    yaxis_title="No of cases",
    font=dict(
        family='Candara',
        size=13,
        color="#7f7f7f"
    )
)

fig.show()

In [31]:
India_covid_data

,Date,Country,Last Update,Confirmed,Deaths,Recovered
430,2020-01-30,India,1/30/20 16:00,1,0,0
491,2020-01-31,India,1/31/2020 23:59,1,0,0
547,2020-02-01,India,1/31/2020 8:15,1,0,0
607,2020-02-02,India,2020-02-02T06:03:08,2,0,0
672,2020-02-03,India,2020-02-03T21:43:02,3,0,0
...,...,...,...,...,...,...
39281,2020-06-10,India,2020-06-11 03:33:41,864,1,192
39295,2020-06-10,India,2020-06-11 03:33:41,9227,0,0
39301,2020-06-10,India,2020-06-11 03:33:41,11335,301,6669
39302,2020-06-10,India,2020-06-11 03:33:41,1537,13,755


In [32]:
import plotly.express as px
config = dict({'scrollZoom': False})

In [33]:
India_fig = px.choropleth(India_covid_data, locations="Country", 
                    locationmode='country names', color="Confirmed", 
                    hover_name="Country", range_color=[1,60000],
                    color_continuous_scale=["green","yellow","red"], 
                    animation_frame="Date",
                    title='India confirmed cases',scope='asia', height=750)
India_fig.show(config=config)

In [34]:
world = go.Figure()

world.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        lon = data["Long_"],
        lat = data["Lat"],
        hovertext = data["text"],
        showlegend=False,
        marker = dict(
            size = 5,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = True,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(50, 50, 50)'
            ),
            cmin = 0,
            color = data['Confirmed'],
            cmax = data['Confirmed'].max(),
            colorbar_title="Confirmed Cases<br>Latest Update",  
            colorbar_x = -0.05
        )

    ),  
)

world.update_geos(
    projection_type="natural earth",
    landcolor="grey",
    oceancolor="LightBlue",
    showocean=True,
    lakecolor="LightBlue",
)

world.update_layout(
    template="plotly_dark",
    title_text = "Global COVID-19 Cases",
    title_x=0.5,
    showlegend=False,
    legend=dict(x=0.65, y = -0.1),
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="#fff",        
    )
)

world.show()